In [1]:
!pip install pyzipcode

     |████████████████████████████████| 1.9MB 5.1MB/s 
  Created wheel for pyzipcode: filename=pyzipcode-2.0.0-cp36-none-any.whl size=1931040 sha256=daddaa37cb2dc45a46efff5ca6a588f3c32246ce26d9269635e4badfa1c800d0
  Stored in directory: /root/.cache/pip/wheels/dd/0b/3d/3769c445fa954e1028d9ce89f54ac8db7d6ffb6fb4f8aeb554
Successfully built pyzipcode


In [2]:
import numpy as np 
import pandas as pd 
from pyzipcode import ZipCodeDatabase

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/jiobu1/Workspace_Datasets/main/Datasets/Census%20Data/us_population_by_zip/population_by_zip_2010.csv?token=ANCU3NVSKQSPRJOZFSNWMGDABHL66", dtype={'zipcode':'category'})
df = df.dropna()
df.head(5)

,population,minimum_age,maximum_age,gender,zipcode,geo_id
0,50,30.0,34.0,female,61747,8600000US61747
2,1389,30.0,34.0,male,95117,8600000US95117
3,231,60.0,61.0,female,74074,8600000US74074
4,56,0.0,4.0,female,58042,8600000US58042
5,524,22.0,24.0,female,75241,8600000US75241


In [4]:
zip_df = pd.read_csv("https://raw.githubusercontent.com/jiobu1/Workspace_Datasets/main/Datasets/Census%20Data/counties%20to%20zip/ZIP-COUNTY-FIPS_2017-06.csv?token=ANCU3NU64O656R6CGG6AD4TABHLZA", dtype={"ZIP":'category'})
zip_df.head(5)

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,36003,Autauga County,AL,1001,H1
1,36006,Autauga County,AL,1001,H1
2,36067,Autauga County,AL,1001,H1
3,36066,Autauga County,AL,1001,H1
4,36703,Autauga County,AL,1001,H1


In [5]:
pop_df = pd.read_csv("https://raw.githubusercontent.com/jiobu1/Workspace_Datasets/main/Datasets/Census%20Data/census_demographic/acs2015_county_data.csv?token=ANCU3NSG2H64KF63GU2BAU3ABHNHS")
pop_df.head(5)



,CensusId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Citizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga,55221,26745,28476,2.6,75.8,18.5,0.4,1.0,0.0,40725,51281.0,2391.0,24974,1080,12.9,18.6,33.2,17.0,24.2,8.6,17.1,87.5,8.8,0.1,0.5,1.3,1.8,26.5,23986,73.6,20.9,5.5,0.0,7.6
1,1003,Alabama,Baldwin,195121,95314,99807,4.5,83.1,9.5,0.6,0.7,0.0,147695,50254.0,1263.0,27317,711,13.4,19.2,33.1,17.7,27.1,10.8,11.2,84.7,8.8,0.1,1.0,1.4,3.9,26.4,85953,81.5,12.3,5.8,0.4,7.5
2,1005,Alabama,Barbour,26932,14497,12435,4.6,46.2,46.7,0.2,0.4,0.0,20714,32964.0,2973.0,16824,798,26.7,45.3,26.8,16.1,23.1,10.8,23.1,83.8,10.9,0.4,1.8,1.5,1.6,24.1,8597,71.8,20.8,7.3,0.1,17.6
3,1007,Alabama,Bibb,22604,12073,10531,2.2,74.5,21.4,0.4,0.1,0.0,17495,38678.0,3995.0,18431,1618,16.8,27.9,21.5,17.9,17.8,19.0,23.7,83.2,13.5,0.5,0.6,1.5,0.7,28.8,8294,76.8,16.1,6.7,0.4,8.3
4,1009,Alabama,Blount,57710,28512,29198,8.6,87.9,1.5,0.3,0.1,0.0,42345,45813.0,3141.0,20532,708,16.7,27.2,28.5,14.1,23.9,13.5,19.9,84.9,11.2,0.4,0.9,0.4,2.3,34.9,22189,82.0,13.5,4.2,0.4,7.7


In [6]:
income_df = pd.read_csv('https://raw.githubusercontent.com/jiobu1/Workspace_Datasets/main/Datasets/Census%20Data/kaggle_incom/kaggle_income.csv?token=ANCU3NTWMURR56TAGID2LDTABHL24',encoding='latin-1')
income_df = income_df.rename(columns={'State_ab':'State'})
income_df.head(5)

,id,State_Code,State_Name,State,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
0,1011000,1,Alabama,AL,Mobile County,Chickasaw,Chickasaw city,City,place,36611,251,10894952,909156,30.771450,-88.079697,38773,30506,33101,1638.260513
1,1011010,1,Alabama,AL,Barbour County,Louisville,Clio city,City,place,36048,334,26070325,23254,31.708516,-85.611039,37725,19528,43789,258.017685
2,1011020,1,Alabama,AL,Shelby County,Columbiana,Columbiana city,City,place,35051,205,44835274,261034,33.191452,-86.615618,54606,31930,57348,926.031000
3,1011030,1,Alabama,AL,Mobile County,Satsuma,Creola city,City,place,36572,251,36878729,2374530,30.874343,-88.009442,63919,52814,47707,378.114619
4,1011040,1,Alabama,AL,Mobile County,Dauphin Island,Dauphin Island,Town,place,36528,251,16204185,413605152,30.250913,-88.171268,77948,67225,54270,282.320328


In [7]:
def zip_finder(CountNum):
    try:
        return (((zip_df[zip_df['STCOUNTYFP']==int(CountNum)])['ZIP']).values)[0]
    except:
        print(CountNum)
        return('None')

In [8]:
def Zip_2_City(zipval):
    zcdb = ZipCodeDatabase()
    if zipval != 'None':
        zipcode = zcdb[zipval]
        if zipcode.city != None:
            return zipcode.city, zipcode.state
        else: return "None",'None'
    else:
        return "None"

In [9]:
def City_State_list_maker(zip_list):   
    City_list = list()
    State_list = list()
    for zips in zip_list:
        if zips == 'None':
            City_list.append('None')
            State_list.append('None')
        else:
            try: 
                city, state = Zip_2_City(zips)
                City_list.append(city)
                State_list.append(state)
            except:
                City_list.append('None')
                State_list.append('None')
    return City_list, State_list

### Income Preparation

In [10]:
Mean_Income_df = income_df[['Mean','Median','Stdev','City','State']].groupby(['City','State'], as_index=False).mean()
Mean_Income_df.to_csv('Mean_Income_df.csv',index=False)
Mean_Income_df.head()


,City,State,Mean,Median,Stdev
0,Abbeville,AL,40518.000000,25216.000000,39126.000000
1,Abbeville,LA,30647.000000,25628.333333,21177.666667
2,Abbeville,SC,45585.666667,32245.000000,41647.333333
3,Abbotsford,WI,58254.000000,44919.000000,48933.000000
4,Aberdeen,MD,96480.000000,80370.000000,64102.000000


### Population -> City and State columns

In [11]:
zip_array = list(df['zipcode'].values)
print(type(zip_array))
print(zip_array[:5])

<class 'list'>
['61747', '95117', '74074', '58042', '75241']


In [12]:
City_list = list()
State_list = list()
City_list, State_list = City_State_list_maker(zip_array)

In [13]:
City_State_df = pd.DataFrame(data={'zipcode':zip_array,
                                  'City':City_list, 'State':State_list})

In [14]:
City_State_df.head()


,zipcode,City,State
0,61747,Hopedale,IL
1,95117,San Jose,CA
2,74074,Stillwater,OK
3,58042,Harwood,ND
4,75241,Dallas,TX


In [15]:
new_df = df.merge(City_State_df, on='zipcode')

In [16]:
new_df = new_df[['population', 'minimum_age','maximum_age','City','State']]
pop_count = new_df[['population','City','State' ]].groupby(['City','State'], as_index=False).sum()
pop_min = new_df[['minimum_age','City','State' ]].groupby(['City','State'], as_index=False).min()
pop_max = new_df[['maximum_age','City','State' ]].groupby(['City','State'], as_index=False).max()

In [17]:
Population_df = pop_count.merge(pop_min, on=['State','City'])
Population_df = Population_df.merge(pop_max, on=['State','City'])
Population_df = Population_df.reset_index()
Population_df.to_csv('Population_df.csv',index=False)
Population_df.head()

,index,City,State,population,minimum_age,maximum_age
0,0,Aaronsburg,PA,52888,0.0,84.0
1,1,Abbeville,AL,289564,0.0,84.0
2,2,Abbeville,GA,208956,0.0,84.0
3,3,Abbeville,LA,1092520,0.0,84.0
4,4,Abbeville,MS,103532,0.0,84.0


### US Demographic Data -> City and State columns

In [18]:
County_names = pop_df['CensusId'].values
zip_list = list()
for i, county_name in enumerate(County_names):
    zip_list.append(zip_finder(county_name))


2158
46102


In [19]:
print(np.where(County_names==2158))
print(np.where(County_names==46102))
zip_list[81] == 'None'
zip_list[2412] == 'None'

(array([81]),)
(array([2412]),)


True

In [20]:
City_list = list()
State_list = list()
City_list, State_list = City_State_list_maker(zip_list)
sum(x is 'None' for x in State_list) == sum(x is 'None' for x in City_list)


True

In [21]:
Zip_and_city_county = pd.DataFrame(data={'City':City_list,'State':State_list, 'Zipcode':zip_list, 'CensusId': County_names})
Zip_and_city_county = Zip_and_city_county.replace(to_replace='None', value=np.nan)

In [22]:
pop_complete_df =  pop_df[pop_df.columns[pop_df.columns != 'State']].merge(Zip_and_city_county, on='CensusId')
print(pop_complete_df.shape)
print(pop_complete_df['Zipcode'].iloc[81])


(3220, 39)
nan


In [23]:
noneless_df = pop_complete_df.dropna()
print(noneless_df.shape)
print(noneless_df['Zipcode'].iloc[81])

(3192, 39)
99579


In [24]:
noneless_df.columns


Index(['CensusId', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic', 'White',
       'Black', 'Native', 'Asian', 'Pacific', 'Citizen', 'Income', 'IncomeErr',
       'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty',
       'Professional', 'Service', 'Office', 'Construction', 'Production',
       'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome',
       'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork', 'SelfEmployed',
       'FamilyWork', 'Unemployment', 'City', 'State', 'Zipcode'],
      dtype='object')

In [25]:
Diversity_df = noneless_df[['Hispanic','White', 'Black', 'Native', 'Asian', 'Pacific','City','State']].groupby(['City','State'],as_index=False).mean()
Diversity_df.to_csv("Diversity_df.csv",index=False)
Diversity_df.head()

,City,State,Hispanic,White,Black,Native,Asian,Pacific
0,Aberdeen,WA,9.6,80.1,1.1,4.0,1.5,0.4
1,Abilene,TX,23.2,65.6,7.2,0.4,1.7,0.0
2,Abingdon,MD,4.0,77.6,12.9,0.2,2.8,0.0
3,Abingdon,VA,1.4,95.6,1.4,0.0,0.6,0.0
4,Adams Run,SC,4.8,65.1,25.2,0.5,1.7,0.1


In [26]:
Income_per_capita_df = noneless_df[['Income','IncomeErr', 'IncomePerCap', 'IncomePerCapErr','City','State']].groupby(['City','State'],as_index=False).mean()
Income_per_capita_df.to_csv("Income_per_capita_df.csv",index=False)
Income_per_capita_df.head()

,City,State,Income,IncomeErr,IncomePerCap,IncomePerCapErr
0,Aberdeen,WA,43538.0,1652.0,22387.0,757.0
1,Abilene,TX,45396.0,1528.0,23896.0,744.0
2,Abingdon,MD,80465.0,1153.0,35746.0,520.0
3,Abingdon,VA,43310.0,1910.0,25351.0,968.0
4,Adams Run,SC,54901.0,1488.0,25223.0,776.0


In [27]:
Poverty_df = noneless_df[['Poverty','ChildPoverty','City','State']].groupby(['City','State'],as_index=False).sum()
Poverty_df.to_csv("Poverty_df.csv",index=False)
Poverty_df.head()

,City,State,Poverty,ChildPoverty
0,Aberdeen,WA,18.1,24.8
1,Abilene,TX,17.4,21.6
2,Abingdon,MD,8.0,11.7
3,Abingdon,VA,12.8,17.1
4,Adams Run,SC,13.1,18.4


In [28]:
Industry_df = noneless_df[['Professional', 'Service', 'Office', 'Construction','Production','City','State']].groupby(['City','State'],as_index=False).sum()
Industry_df.to_csv("Industry_df.csv",index=False)
Industry_df.head()

,City,State,Professional,Service,Office,Construction,Production
0,Aberdeen,WA,25.0,24.9,23.1,11.8,15.1
1,Abilene,TX,30.3,21.9,25.8,11.1,10.9
2,Abingdon,MD,42.4,14.7,25.0,8.6,9.3
3,Abingdon,VA,34.4,17.1,24.8,7.3,16.4
4,Adams Run,SC,34.5,16.7,26.1,10.5,12.3


In [29]:
Trasportation_df = noneless_df[['Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp','WorkAtHome', 'MeanCommute','City','State']].groupby(['City','State'],as_index=False).mean()
Trasportation_df.to_csv("Trasportation_df.csv",index=False)
Trasportation_df.head()

,City,State,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute
0,Aberdeen,WA,76.9,13.4,1.5,2.2,2.0,3.9,24.2
1,Abilene,TX,81.4,11.4,0.6,2.3,1.8,2.5,15.9
2,Abingdon,MD,83.2,9.1,1.7,1.4,0.8,3.9,31.7
3,Abingdon,VA,84.2,8.8,0.3,1.8,0.6,4.2,23.2
4,Adams Run,SC,83.6,9.9,0.6,0.9,1.2,3.8,28.2


In [30]:
Employment_df = noneless_df[['Employed','Unemployment','City','State']].groupby(['City','State'],as_index=False).sum()
Employment_df.to_csv('Employment_df.csv',index=False)
Employment_df.head()

,City,State,Employed,Unemployment
0,Aberdeen,WA,26197,14.3
1,Abilene,TX,60708,5.9
2,Abingdon,MD,127274,6.4
3,Abingdon,VA,24442,6.9
4,Adams Run,SC,66723,8.0


In [31]:
Employment_ratio_df = noneless_df[['PrivateWork', 'PublicWork','SelfEmployed', 'FamilyWork','City','State']].groupby(['City','State'],as_index=False).mean()
Employment_ratio_df.to_csv('Employment_ratio_df.csv',index=False)
Employment_ratio_df.head()

,City,State,PrivateWork,PublicWork,SelfEmployed,FamilyWork
0,Aberdeen,WA,71.2,21.7,7.0,0.2
1,Abilene,TX,77.5,16.5,5.7,0.2
2,Abingdon,MD,74.4,21.2,4.2,0.2
3,Abingdon,VA,78.1,15.9,6.0,0.0
4,Adams Run,SC,77.0,18.3,4.5,0.2


In [32]:
total_df = Population_df.merge(Diversity_df, on=['State','City'])
total_df = total_df.merge(Poverty_df, on=['State','City'])
total_df = total_df.merge(Industry_df, on=['State','City'])
total_df = total_df.merge(Employment_df, on=['State','City'])
total_df = total_df.merge(Trasportation_df, on=['State','City'])
total_df = total_df.merge(Employment_ratio_df, on=['State','City'])
total_df = total_df.merge(Income_per_capita_df, on=['State','City'])
total_df = total_df.merge(Mean_Income_df, on=['State','City'])
total_df.to_csv('Total_Search_df.csv',index=False)
total_df.head(5)

,index,City,State,population,minimum_age,maximum_age,Hispanic,White,Black,Native,Asian,Pacific,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Employed,Unemployment,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Mean,Median,Stdev
0,19,Aberdeen,WA,1029028,0.0,84.0,9.6,80.1,1.1,4.0,1.5,0.4,18.1,24.8,25.0,24.9,23.1,11.8,15.1,26197,14.3,76.9,13.4,1.5,2.2,2.0,3.9,24.2,71.2,21.7,7.0,0.2,43538.0,1652.0,22387.0,757.0,72647.000000,80209.000000,46242.000000
1,24,Abilene,TX,5365712,0.0,84.0,23.2,65.6,7.2,0.4,1.7,0.0,17.4,21.6,30.3,21.9,25.8,11.1,10.9,60708,5.9,81.4,11.4,0.6,2.3,1.8,2.5,15.9,77.5,16.5,5.7,0.2,45396.0,1528.0,23896.0,744.0,51258.923077,79425.538462,38462.769231
2,26,Abingdon,MD,1299364,0.0,84.0,4.0,77.6,12.9,0.2,2.8,0.0,8.0,11.7,42.4,14.7,25.0,8.6,9.3,127274,6.4,83.2,9.1,1.7,1.4,0.8,3.9,31.7,74.4,21.2,4.2,0.2,80465.0,1153.0,35746.0,520.0,109797.000000,300000.000000,57646.500000
3,27,Abingdon,VA,1152008,0.0,84.0,1.4,95.6,1.4,0.0,0.6,0.0,12.8,17.1,34.4,17.1,24.8,7.3,16.4,24442,6.9,84.2,8.8,0.3,1.8,0.6,4.2,23.2,78.1,15.9,6.0,0.0,43310.0,1910.0,25351.0,968.0,60952.500000,47182.500000,54058.000000
4,109,Adena,OH,94556,0.0,84.0,0.8,93.2,3.8,0.2,0.4,0.0,14.6,21.5,26.4,19.9,23.7,14.6,15.3,29728,8.5,86.3,7.7,0.4,2.6,1.1,1.9,22.5,82.2,13.9,3.8,0.1,43833.0,1491.0,23684.0,711.0,51340.500000,48046.500000,37181.500000


In [33]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [34]:
df.to_csv('total_df.csv')

In [37]:
!cp total_df.csv "drive/My Drive/"